# Deep Nets

TensorFlow has many abstractions: higher level API to simplify the code and it usually allows skipping things, at the cost of control.

Many of these abstractions reside in TensorFlow's tf.contrib section. Typically, libraries get developed and accepted into contrib and then 'graduate' to being included as part of the standard TensorFlow library. The speed of development of TensorFlow has caused abstractions to come and go quickly.

This series of lectures will focus on presenting the most common abstractions used:

- Estimator API
- Keras
- Layers

We will focus on understanding how to use these abstractions to build deep densely connected neural networks. Using these abstractions makes it easy to stack layers on top of each other for simpler tasks.

Let's start by exploring the dataset.

In [1]:
from sklearn.datasets import load_wine
wine_data = load_wine()

In [2]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [3]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [4]:
feat_data = wine_data['data']

In [5]:
labels = wine_data['target']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size = 0.3, random_state=101)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.fit_transform(X_test)

# Layers

In [10]:
import tensorflow as tf

/home/alvaro/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Layers is separated as contrib and within the tensorflow library

In [11]:
import pandas as pd

In [14]:
onehot_y_train = pd.get_dummies(y_train).as_matrix()

In [16]:
onehot_y_train

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0,

In [17]:
onehot_y_test = pd.get_dummies(y_test).as_matrix()

In [24]:
num_features = 13
num_hidden1=13
num_hidden2=13
num_outputs=3

learning_rate= 0.01

In [25]:
from tensorflow.contrib.layers import fully_connected

In [26]:
X = tf.placeholder(tf.float32, shape=[None, num_features])
y_true = tf.placeholder(tf.float32, shape=[None, num_outputs])

In [27]:
actf = tf.nn.relu

In [28]:
hidden1 = fully_connected(X, num_hidden1, activation_fn=actf)

In [29]:
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn=actf)

In [31]:
output = fully_connected(hidden2, num_outputs)

In [32]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

In [33]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [34]:
train = optimizer.minimize(loss)

In [35]:
init = tf.global_variables_initializer()

In [38]:
training_steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train, feed_dict={X:scaled_x_train, y_true:onehot_y_train})
        
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits, axis=1) # the class with the most probability
    
    results = preds.eval()

In [39]:
from sklearn.metrics import classification_report

In [40]:
print(classification_report(results, y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       1.00      0.96      0.98        23
          2       0.92      1.00      0.96        12

avg / total       0.98      0.98      0.98        54

